In [1]:
from utils import progress_bar
import numpy as np
from iotools import loader_factory
import matplotlib.pyplot as plt
%matplotlib inline

def Create_Sum_Rings( DATA_DIRS=['/data/hkml_data/IWCDgrid/varyE/e-'], flavour='100k.h5', 
                      output_file='/data/jamieson/data/e-_tworing.npz' ):

                     
    loader=loader_factory('H5Dataset', batch_size=2, 
                          shuffle=True, num_workers=4, data_dirs=DATA_DIRS, 
                          flavour=flavour, start_fraction=0.0, use_fraction=1.0,
                          read_keys=[ 'event_data', 'labels','directions', 'energies', 'pids', 'positions'])    
    #output lists
    ev_data=[]
    labels=[]
    pids=[]
    positions=[]
    directions=[]
    energies=[]
    
    #charge and time data in tqev1,tqev2,tqev3
    tqev1=[]
    tqev2=[]
    tqev3=[]
    progress = display(progress_bar(0,len(loader)),display_id=True)
    for i,evt in enumerate(loader):    
        if ( i==0 or (i+1)%100==0) :
            progress.update( progress_bar(i+1,len(loader),"Progress"))
        tqev1 = evt[0][0]
        tqev2 = evt[0][1]
        tqev3 = np.copy( tqev2 )                                # make a copy of event_data to tqev3
        tqev3[:,:,0] = tqev1[:,:,0] + tqev2[:,:,0]              # add charges
        tqev3[:,:,1] = np.maximum( tqev1[:,:,1], tqev2[:,:,1])  # take maximum time
        
        ev_data.append( np.expand_dims( tqev3, axis=0 ) )
        labels.append(     [ evt[1][0], evt[1][1] ] )
        directions.append( [ evt[2][0], evt[2][1] ] )  
        energies.append(   [ evt[3][0], evt[3][1] ] )  
        pids.append(       [ evt[4][0], evt[4][1] ] )  
        positions.append(  [ evt[5][0], evt[5][1] ] )  
        
    print("Create_Sum_Rings finished loop over events")  
    all_events=np.concatenate(ev_data)
    all_labels=np.asarray(labels)
    all_pids=np.asarray(pids)
    all_positions=np.asarray(positions)
    all_directions=np.asarray(directions)
    all_energies=np.asarray(energies)
    print("Begin data compress")
    np.savez_compressed(output_file,
                    event_data=all_events,
                    labels=all_labels,
                    pids=all_pids,
                    positions=all_positions,
                    directions=all_directions,
                    energies=all_energies)
    print("Done Create_Sum_Rings")


In [ ]:
Create_Sum_Rings()
print("Done.")